## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import json
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
#city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,82.40,65,20,18.41,few clouds
1,1,Kodiak,US,57.7900,-152.4072,30.20,34,1,4.61,clear sky
2,2,Port Alfred,ZA,-33.5906,26.8910,63.14,82,100,17.94,overcast clouds
3,3,Nishihara,JP,26.1842,127.7558,71.60,94,20,4.61,few clouds
4,4,Severo-Kurilsk,RU,50.6789,156.1250,27.73,82,60,17.11,broken clouds


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,82.40,65,20,18.41,few clouds
9,9,Tonneins,FR,44.3921,0.3124,75.20,35,0,3.44,clear sky
10,10,Rikitea,PF,-23.1203,-134.9692,76.71,79,89,12.84,overcast clouds
11,11,Impfondo,CG,1.6381,18.0667,83.14,65,87,0.96,overcast clouds
13,13,Kieta,PG,-6.2167,155.6333,79.90,81,100,0.72,overcast clouds
16,16,Rundu,NaN,-17.9333,19.7667,79.38,43,83,4.29,broken clouds
24,24,Vaini,TO,-21.2000,-175.2000,75.20,94,40,2.30,light rain
26,26,Carnarvon,AU,-24.8667,113.6333,80.60,78,15,12.66,few clouds
30,30,Trairi,BR,-3.2778,-39.2689,87.24,58,35,12.37,scattered clouds
33,33,Bonthe,SL,7.5264,-12.5050,85.82,66,76,12.12,broken clouds


In [60]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                235
City                   235
Country                234
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
dtype: int64

In [61]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [62]:
clean_df.count()

City_ID                234
City                   234
Country                234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [67]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,82.40,few clouds,-22.9661,-42.0278,
9,Tonneins,FR,75.20,clear sky,44.3921,0.3124,
10,Rikitea,PF,76.71,overcast clouds,-23.1203,-134.9692,
11,Impfondo,CG,83.14,overcast clouds,1.6381,18.0667,
13,Kieta,PG,79.90,overcast clouds,-6.2167,155.6333,
16,Rundu,NaN,79.38,broken clouds,-17.9333,19.7667,
24,Vaini,TO,75.20,light rain,-21.2000,-175.2000,
26,Carnarvon,AU,80.60,few clouds,-24.8667,113.6333,
30,Trairi,BR,87.24,scattered clouds,-3.2778,-39.2689,
33,Bonthe,SL,85.82,broken clouds,7.5264,-12.5050,


In [69]:

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

Hotel not found... skipping.


In [70]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,82.40,few clouds,-22.9661,-42.0278,Pousada Porto Praia
9,Tonneins,FR,75.20,clear sky,44.3921,0.3124,Hôtel Des Fleurs SARL FLOWERS
10,Rikitea,PF,76.71,overcast clouds,-23.1203,-134.9692,Pension Maro'i
11,Impfondo,CG,83.14,overcast clouds,1.6381,18.0667,Préfecture de la Likouala
13,Kieta,PG,79.90,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
...,...,...,...,...,...,...,...
690,Osmena,PH,82.22,overcast clouds,10.5208,119.9258,Arjemarkyza Lodge
692,Conceicao Do Araguaia,BR,85.71,broken clouds,-8.2578,-49.2647,Tarumã Tropical Hotel
694,Daru,PG,75.61,overcast clouds,-9.0763,143.2092,Daru Lodge
695,Puerto Escondido,MX,89.73,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa


In [72]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [76]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [78]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))